# Modele Bazowe - Regresja Liniowa i Prophet

Ten notebook implementuje modele bazowe dla prognozowania popytu, które będą służyć jako punkt odniesienia dla modeli neuronowych.

## Modele bazowe:
- Regresja liniowa z cechami czasowymi
- Prophet (model Facebook do szeregów czasowych)
- Średnia ruchoma
- Naiwny model sezonowy

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet import Prophet
import joblib
import sys
sys.path.append('../src')

## 1. Ładowanie przetworzonych danych

In [ ]:
# Ładowanie danych z etapu preprocessing

## 2. Model regresji liniowej

In [ ]:
# Implementacja i trenowanie regresji liniowej

## 3. Model Prophet

In [ ]:
# Przygotowanie danych dla Prophet (ds, y)

In [ ]:
# Trenowanie modelu Prophet

In [ ]:
# Generowanie prognoz Prophet

## 4. Modele naiwne

In [ ]:
# Średnia ruchoma i model sezonowy

## 5. Ewaluacja modeli bazowych

In [ ]:
# Obliczenie metryk MAE, RMSE, R²

## 6. Wizualizacja wyników

In [ ]:
# Wykresy porównawcze prognoz vs rzeczywiste wartości

## 7. Zapisanie modeli i wyników

In [ ]:
# Zapis wytrenowanych modeli i metryk